# Prepare SILVER



In [3]:
# Spark Config

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "?sv=2021-12-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2023-08-29T21:29:15Z&st=2023-03-23T12:29:15Z&spr=https&sig=ghlTFjtWaYvdXVmKQJj8uM5gt4C5P9bhuuRsoo7Y4sk%3D")

storage_account_name = "stgd365analytics"


In [10]:
from pyspark.sql.functions import col
from datetime import datetime

dataverse_customer_table_name = 'crc33_customer'
customer_spend = 'customer_spend'
silver_container_name = 'silver'
gold_customer_table_name = 'customer'
gold_container_name = 'gold'
partition_date = datetime.now().strftime("%Y-%m-%d")

customer_spend_df = spark.read.parquet(f"abfss://{silver_container_name}@{storage_account_name}.dfs.core.windows.net/{customer_spend}")
dataverse_customer_df = spark.read.parquet(f"abfss://{silver_container_name}@{storage_account_name}.dfs.core.windows.net/{dataverse_customer_table_name}")

customer_df = dataverse_customer_df.join(customer_spend_df,dataverse_customer_df.cid ==  customer_spend_df.cid,"inner").drop(customer_spend_df.cid)

customer_df.write.mode('append').parquet(f"abfss://{gold_container_name}@{storage_account_name}.dfs.core.windows.net/{gold_customer_table_name}")